In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("data/all.csv", header = None)
df.columns = ["student", "task", "percentile"]

In [13]:
# Get one random test from each user
size = 1        
replace = False  
fn = lambda obj: obj.loc[np.random.choice(obj.index, size, replace),:]
test = df.groupby("student", as_index=False).apply(fn)
test.index = [a[1] for a in test.index]

In [31]:
outer_join = df.merge(test, how = 'outer', indicator = True)
train = outer_join[~(outer_join._merge == 'both')].drop('_merge', axis = 1)

In [35]:
train.to_csv("data/train.csv", header=False, index=False)
test.to_csv("data/test.csv", header=False, index=False)

In [7]:
size = 1
replace = False
fn = lambda obj: np.random.choice(obj.index, size, replace)
[a[0] for a in df.groupby("student", as_index=False).apply(fn).values]

[8,
 13,
 20,
 27,
 42,
 53,
 56,
 65,
 74,
 83,
 94,
 99,
 110,
 117,
 130,
 141,
 144,
 159,
 168,
 176,
 187,
 196,
 198,
 214,
 216,
 225,
 238,
 246,
 253,
 267,
 278,
 287,
 290,
 300,
 313,
 319,
 328,
 334,
 348,
 358,
 361,
 371,
 380,
 395,
 399,
 413,
 421,
 423,
 433,
 446,
 450,
 464,
 470,
 484,
 493,
 496,
 506,
 514,
 529,
 536,
 544,
 552,
 566,
 575,
 580,
 587,
 600,
 603,
 616,
 620,
 633,
 635,
 645,
 655,
 664,
 668,
 674,
 684,
 695,
 700,
 710,
 721,
 722,
 731,
 745,
 751,
 755,
 767,
 771,
 783,
 787,
 799,
 806,
 814,
 823,
 830,
 838,
 844,
 851,
 864,
 868,
 879,
 885,
 895,
 905,
 906,
 921,
 927,
 935,
 942,
 948,
 961,
 964,
 977,
 980,
 992,
 997,
 1008,
 1016,
 1027,
 1038,
 1044,
 1054,
 1064,
 1072,
 1082,
 1092,
 1102,
 1110,
 1117,
 1126,
 1138,
 1139,
 1156,
 1165,
 1166,
 1179,
 1191,
 1193,
 1203,
 1219,
 1221,
 1232,
 1241,
 1248,
 1261,
 1269,
 1274,
 1283,
 1292,
 1309,
 1314,
 1326,
 1334,
 1343,
 1350,
 1356,
 1365,
 1373,
 1390,
 1396,
 14